## Carga de datos

Antes de poder manipular los diferentes datos, primero es necesário
cargarlos en el sistema

In [1]:
import pandas as pd
import polars as pl
import numpy as np 
from pathlib import Path
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [2]:
# Directório de archivos
data_folder = Path("../data/predict-energy-behavior-of-prosumers")

# Diccionario de archivos
files = {
    "train": "train.csv",
    "gas_prices": "gas_prices.csv",
    "client": "client.csv",
    "electricity_prices": "electricity_prices.csv",
    "forecast_weather": "forecast_weather.csv",
    "historical_weather": "historical_weather.csv",
    "county_lon_lats": "county_lon_lats.csv",
    "weather_station": "weather_station_to_county_mapping.csv"
}

In [3]:
# Se crea un diccionario de DataFrames para acceder facilmente a todos
# y poder iterar sobre ellos si es necesário
dfs = {}

for key, filename in files.items():
    filepath = data_folder / filename
    try:
        df = pl.read_csv(filepath)
        dfs[key] = df
        print(f"Archivo {key} cargado con exito.")
    except:
        print(f"No pudo cargarse el archivo {key}.")
    

Archivo train cargado con exito.
Archivo gas_prices cargado con exito.
Archivo client cargado con exito.
Archivo electricity_prices cargado con exito.
Archivo forecast_weather cargado con exito.
Archivo historical_weather cargado con exito.
Archivo county_lon_lats cargado con exito.
Archivo weather_station cargado con exito.


## Asignación de longitud/latitud a un condado especifico

Con el fin de poder unir los csv de "train.csv" y "forecast_weather.csv"
es necesario poder asignar a cada longitud y latitud, su condado
correspondiente.

Para ello se utilizará el archivo "weather_station_to_county_mapping" con datos que asignan latitudes y longitudes a su correspondiente condado.

### Visualización de los datos

In [4]:
from geopy.geocoders import Nominatim

In [4]:
# Se eliminan las filas con instancias nulas
weather_station = dfs["weather_station"].drop_nulls()
weather_station.head()

county_name,longitude,latitude,county
str,f64,f64,i64
"""Saaremaa""",22.2,58.2,10
"""Saaremaa""",22.2,58.5,10
"""Saaremaa""",22.7,58.5,10
"""Hiiumaa""",22.7,58.8,1
"""Saaremaa""",23.2,58.5,10


In [5]:
coords1 = dfs["historical_weather"]["latitude", "longitude"]

In [13]:
coords2 = dfs["weather_station"]["latitude", "longitude"]

In [11]:
coords1 = coords1.unique()

In [29]:
coords1.with_columns(pl.col("latitude").radians().alias("lat_rad"))

latitude,longitude,lat_rad
f64,f64,f64
58.2,25.7,1.015782
58.2,24.2,1.015782
58.8,26.7,1.026254
59.1,23.2,1.03149
57.9,28.2,1.010546
…,…,…
59.1,26.7,1.03149
58.5,25.7,1.021018
58.8,24.7,1.026254


In [12]:
lat1[0] = 22
lat1

latitude
f64
22.0
57.6
57.6
57.6
57.6
…
59.7
59.7
59.7


In [6]:
import re

class Regex_dict:
    '''
    Diccionario el cual almacena expresiones regulares como keys
    con el fin de poder compilar la entrada a la hora de obtener un value
    '''
    def __init__(self):
        self.keywords = []  # Expresiones regulares

    def add(self, expr, value):
        '''
        Almacena la expresión regular junto a su valor.
        '''
        self.keywords.append(
            (re.compile(expr), value)
        )
    
    def get(self, string):
        '''
        Devuelve, si es posible, el valor de la key del string pasado
        '''
        for expr, value in self.keywords:
            if expr.search(string):
                return value
        return None

In [8]:
# Se inicializa el diccionario de er
county_dict = Regex_dict()

# se obtiene un array con el nombre de los condados y su número asignado
c = ("county_name", "county")
counties = weather_station[c].unique().to_numpy()

In [9]:
"counties" in weather_station and "county_name" in weather_station

False

In [9]:
# Se añaden los nombres y número de los condados al diccionario
for county_name, county in counties:
    county_dict.add(county_name, county)

In [10]:
county_dict.keywords

[(re.compile(r'Võrumaa', re.UNICODE), 15),
 (re.compile(r'Lääne-Virumaa', re.UNICODE), 5),
 (re.compile(r'Ida-Virumaa', re.UNICODE), 2),
 (re.compile(r'Hiiumaa', re.UNICODE), 1),
 (re.compile(r'Jõgevamaa', re.UNICODE), 4),
 (re.compile(r'Valgamaa', re.UNICODE), 13),
 (re.compile(r'Raplamaa', re.UNICODE), 9),
 (re.compile(r'Järvamaa', re.UNICODE), 3),
 (re.compile(r'Viljandimaa', re.UNICODE), 14),
 (re.compile(r'Harjumaa', re.UNICODE), 0),
 (re.compile(r'Tartumaa', re.UNICODE), 11),
 (re.compile(r'Pärnumaa', re.UNICODE), 7),
 (re.compile(r'Põlvamaa', re.UNICODE), 8),
 (re.compile(r'Saaremaa', re.UNICODE), 10),
 (re.compile(r'Läänemaa', re.UNICODE), 6)]

In [23]:
coords = dfs["weather_station"]["latitude", "longitude"].to_numpy()

county_list = np.array([])
'''for coord in coords:
    location = geolocator.reverse((coord[0], coord[1]), exactly_one=True)
    county_name = location.raw["address"].get("municipality")
    county_list = np.append(
        county_list,
        self.county_dict.get(county_name)
    )'''
lats, long = coords[1, 0], coords[1, 1]
loc = geolocator.reverse((lats, long), exactly_one=True)
addrs = loc.raw["address"]

In [24]:
lats, long

(np.float64(57.9), np.float64(21.7))

In [29]:
prueba = dfs["historical_weather"].join(
    dfs["weather_station"],
    how="left",
    on=["latitude", "longitude"]
)
prueba

datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,county_name,county
str,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,i64,f64,f64,f64,f64,f64,f64,str,i64
"""2021-09-01 00:00:00""",14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0,null,null
"""2021-09-01 00:00:00""",13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.111111,359,0.0,0.0,0.0,57.6,22.2,1.0,null,null
"""2021-09-01 00:00:00""",14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.333333,355,0.0,0.0,0.0,57.6,22.7,1.0,null,null
"""2021-09-01 00:00:00""",14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,57.6,23.2,1.0,null,null
"""2021-09-01 00:00:00""",15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.416667,5,0.0,0.0,0.0,57.6,23.7,1.0,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-05-30 10:00:00""",11.7,4.6,0.0,0.0,1018.9,40,9,54,0,1.055556,253,567.0,392.0,175.0,59.7,26.2,637.0,null,null
"""2023-05-30 10:00:00""",12.3,3.5,0.0,0.0,1019.0,46,4,70,0,0.805556,263,581.0,407.0,174.0,59.7,26.7,637.0,null,null
"""2023-05-30 10:00:00""",9.8,3.0,0.0,0.0,1019.2,41,4,62,0,1.972222,285,609.0,432.0,177.0,59.7,27.2,637.0,null,null


In [25]:
class DataProcessor:
    def __init__(self):
        self.gas_join = ["datetime"]
        self.ep_join = ["datetime"]
        self.hw_join = [ "county", "datetime"]
        self.client_join = ["product_type", "county", "is_business", "date"]
        self.lat_lon = ["latitude", "longitude"]

    def create_dict(self, lon_lat_dataframe: pl.DataFrame):
        # Se inicializa el diccionario de re
        re_dict = Regex_dict()

        # Se eliminan los valores núlos del dataframe
        lon_lat_dataframe = lon_lat_dataframe.drop_nulls()
        cols = "county_name", "county"

        if (cols[0] in lon_lat_dataframe) and (cols[1] in lon_lat_dataframe):
            # Se obtiene un array con los nombres y números de cada condado
            counties = lon_lat_dataframe[cols].unique().to_numpy()

            # Se añade los nombres y números al diccionario
            for county_name, county in counties:
                re_dict.add(county_name, county)
        
        else:
            raise RuntimeError(
                "No existen las columnas \"county_name\" y/o \"county\" en el dataframe"
            )

        return re_dict

    def add_counties(self, df: pl.DataFrame):
        geolocator = Nominatim(user_agent="get-county")
        coords = df["latitude", "longitude"].to_numpy()

        county_list = np.array([])
        for coord in coords:
            location = geolocator.reverse((coord[0], coord[1]), exactly_one=True)
            county_name = location.raw["address"].get("municipality")
            county_list = np.append(
                county_list,
                self.county_dict.get(county_name)
            )
        
        return df.with_columns(pl.Series(name="county", values=county_list))



    def change_names(self, df: pl.DataFrame, suffix, no_change):
        renamed_df = df.rename(
            lambda col: col + suffix if col not in no_change else col
        )
        return renamed_df
    
    def to_datetime(self, df: pl.DataFrame, col: str):
        # Cast str to Datetime
        result = df.with_columns(
            pl.col(col).str.strptime(pl.Datetime)
        )
        return result

    def get_gas_features(self, gas_df: pl.DataFrame):
        df = gas_df.with_columns(
            ((pl.col("lowest_price_per_mwh") + 
              pl.col("highest_price_per_mwh")) / 2).alias("mean_price_per_mwh_24h")
        )

        df = self.to_datetime(df, "origin_date")
        df = df.rename({"origin_date":"datetime"})

        return self.change_names(df, "_gas", self.gas_join)
    
    def get_client_features(self, client_df: pl.DataFrame):
        df = client_df.with_columns(
            pl.col("date").str.strptime(pl.Date)
        )
        return self.change_names(df, "_client", self.client_join)
    
    def get_hist_weather_features(self, hw_df: pl.DataFrame):
        df = self.to_datetime(hw_df, "datetime")

        df = self.add_counties(df)
        df = self.change_names(df, "_hw", self.hw_join)

        return df
    
    def get_forecast_weather_features(self, fw_df):
        return None
    
    def get_electricity_features(self, ep_df: pl.DataFrame):
        # str to datetime
        df = self.to_datetime(ep_df, "origin_date")

        df = df.rename({"origin_date":"datetime"})
        df = df.rename({"euros_per_mwh":"euros_per_mwh_24h"})

        # Change names
        df = self.change_names(df, "_ep", self.ep_join)

        return df
    
    def get_data_features(self, data: pl.DataFrame):
        
        date = "datetime"
        # Cast to Datetime
        df = self.to_datetime(data, "datetime")

        df = df.with_columns([
            # Date
            pl.col(date).dt.truncate("1d").cast(pl.Date).alias("date"),

            # Year
            pl.col(date).dt.year().alias("year"),

            # Quarter
            pl.col(date).dt.quarter().alias("quarter"),

            # Month
            pl.col(date).dt.month().alias("month"),

            # Week
            pl.col(date).dt.week().alias("week"),

            # Hour
            pl.col(date).dt.hour().alias("hour"),

            # Day of year
            pl.col(date).dt.ordinal_day().alias("day_of_year"),

            # Day of month
            pl.col(date).dt.day().alias("day_of_month"),

            # Day of week
            pl.col(date).dt.weekday().alias("day_of_week")
        ])

        return df
    
    def __call__(self, df_dict: dict):
        
        # Counties dict
        self.county_dict = self.create_dict(df_dict["weather_station"])
        display(self.county_dict.keywords)

        data = self.get_data_features(df_dict["train"])
        gas_p = self.get_gas_features(df_dict["gas_prices"])
        el_p = self.get_electricity_features(df_dict["electricity_prices"])
        client = self.get_client_features(df_dict["client"])
        hist_w = self.get_hist_weather_features(df_dict["historical_weather"])

        final_data = data.join(client, on=self.client_join, how="left")
        final_data = final_data.join(gas_p, on=self.gas_join, how="left")
        final_data = final_data.join(el_p, on=self.ep_join, how="left")
        final_data = final_data.join(hist_w, on=self.hw_join, how="left")

        return final_data

In [26]:
dp = DataProcessor()
dct = dp.create_dict(dfs["weather_station"])

array([[57.6, 21.7],
       [57.6, 22.2],
       [57.6, 22.7],
       ...,
       [59.7, 27.2],
       [59.7, 27.7],
       [59.7, 28.2]])

In [48]:
dfs["weather_station"]

county_name,longitude,latitude,county
str,f64,f64,i64
null,21.7,57.6,null
null,21.7,57.9,null
null,21.7,58.2,null
null,21.7,58.5,null
null,21.7,58.8,null
…,…,…,…
null,28.2,58.5,null
null,28.2,58.8,null
null,28.2,59.1,null


In [24]:
dp = DataProcessor()

result = dp(dfs)
result

[(re.compile(r'Põlvamaa', re.UNICODE), 8),
 (re.compile(r'Läänemaa', re.UNICODE), 6),
 (re.compile(r'Järvamaa', re.UNICODE), 3),
 (re.compile(r'Valgamaa', re.UNICODE), 13),
 (re.compile(r'Lääne-Virumaa', re.UNICODE), 5),
 (re.compile(r'Jõgevamaa', re.UNICODE), 4),
 (re.compile(r'Harjumaa', re.UNICODE), 0),
 (re.compile(r'Pärnumaa', re.UNICODE), 7),
 (re.compile(r'Viljandimaa', re.UNICODE), 14),
 (re.compile(r'Võrumaa', re.UNICODE), 15),
 (re.compile(r'Ida-Virumaa', re.UNICODE), 2),
 (re.compile(r'Raplamaa', re.UNICODE), 9),
 (re.compile(r'Hiiumaa', re.UNICODE), 1),
 (re.compile(r'Tartumaa', re.UNICODE), 11),
 (re.compile(r'Saaremaa', re.UNICODE), 10)]

TypeError: expected string or bytes-like object, got 'NoneType'

In [8]:
dfs["historical_weather"]

NameError: name 'dfs' is not defined

In [11]:
# Adivinar que hace la variable hours ahead f mean
# No añadir los Targets de horas anteriores

In [31]:
dfs["client"].tail()

product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
i64,i64,i64,f64,i64,str,i64
1,15,51,415.6,0,"""2023-05-29""",637
3,15,161,2035.75,0,"""2023-05-29""",637
0,15,15,620.0,1,"""2023-05-29""",637
1,15,20,624.5,1,"""2023-05-29""",637
3,15,55,2188.2,1,"""2023-05-29""",637


In [41]:
display(dfs["train"])
display(dfs["electricity_prices"])

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,str,i64,i64,i64
0,0,1,0.713,0,"""2021-09-01 00:00:00""",0,0,0
0,0,1,96.59,1,"""2021-09-01 00:00:00""",0,1,0
0,0,2,0.0,0,"""2021-09-01 00:00:00""",0,2,1
0,0,2,17.314,1,"""2021-09-01 00:00:00""",0,3,1
0,0,3,2.904,0,"""2021-09-01 00:00:00""",0,4,2
…,…,…,…,…,…,…,…,…
15,1,0,197.233,1,"""2023-05-31 23:00:00""",637,2018347,64
15,1,1,0.0,0,"""2023-05-31 23:00:00""",637,2018348,59
15,1,1,28.404,1,"""2023-05-31 23:00:00""",637,2018349,59


forecast_date,euros_per_mwh,origin_date,data_block_id
str,f64,str,i64
"""2021-09-01 00:00:00""",92.51,"""2021-08-31 00:00:00""",1
"""2021-09-01 01:00:00""",88.9,"""2021-08-31 01:00:00""",1
"""2021-09-01 02:00:00""",87.35,"""2021-08-31 02:00:00""",1
"""2021-09-01 03:00:00""",86.88,"""2021-08-31 03:00:00""",1
"""2021-09-01 04:00:00""",88.43,"""2021-08-31 04:00:00""",1
…,…,…,…
"""2023-05-30 19:00:00""",82.1,"""2023-05-29 19:00:00""",637
"""2023-05-30 20:00:00""",150.85,"""2023-05-29 20:00:00""",637
"""2023-05-30 21:00:00""",82.1,"""2023-05-29 21:00:00""",637


In [ ]:
for key in dfs.keys():
    print(f"{key}:")
    display(dfs[key].head(1))

train:


county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,str,i64,i64,i64
0,0,1,0.713,0,"""2021-09-01 00:00:00""",0,0,0


gas_prices:


forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
str,f64,f64,str,i64
"""2021-09-01""",45.23,46.32,"""2021-08-31""",1


client:


...

datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
str,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,i64,f64,f64,f64,f64,f64,f64
"""2021-09-01 00:00:00""",14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0


county_lon_lats:


,county,longitude,latitude
i64,i64,f64,f64
0,0,24.2,59.1


weather_station:


county_name,longitude,latitude,county
str,f64,f64,i64
null,21.7,57.6,null


In [ ]:
dfs["client"].with_columns(
    pl.col("date").str.strptime(pl.Date)
)
.0

product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
i64,i64,i64,f64,i64,date,i64
1,0,108,952.89,0,2021-09-01,2
2,0,17,166.4,0,2021-09-01,2
3,0,688,7207.88,0,2021-09-01,2
0,0,5,400.0,1,2021-09-01,2
1,0,43,1411.0,1,2021-09-01,2
…,…,…,…,…,…,…
1,15,51,415.6,0,2023-05-29,637
3,15,161,2035.75,0,2023-05-29,637
0,15,15,620.0,1,2023-05-29,637


In [17]:
display(dfs["client"].filter(
    (pl.col("product_type") == 1) & 
    (pl.col("county") == 0) & 
    (pl.col("is_business") == 0)
))

product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
i64,i64,i64,f64,i64,str,i64
1,0,108,952.89,0,"""2021-09-01""",2
1,0,108,952.89,0,"""2021-09-02""",3
1,0,108,952.89,0,"""2021-09-03""",4
1,0,108,952.89,0,"""2021-09-04""",5
1,0,108,952.89,0,"""2021-09-05""",6
…,…,…,…,…,…,…
1,0,508,4968.215,0,"""2023-05-25""",633
1,0,507,4960.215,0,"""2023-05-26""",634
1,0,507,4960.215,0,"""2023-05-27""",635


In [34]:
dfs["train"].filter(
    pl.col("data_block_id") == 1
)

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,str,i64,i64,i64
0,0,1,1.687,0,"""2021-09-02 00:00:00""",1,2928,0
0,0,1,109.366,1,"""2021-09-02 00:00:00""",1,2929,0
0,0,2,0.0,0,"""2021-09-02 00:00:00""",1,2930,1
0,0,2,21.008,1,"""2021-09-02 00:00:00""",1,2931,1
0,0,3,1.003,0,"""2021-09-02 00:00:00""",1,2932,2
…,…,…,…,…,…,…,…,…
15,0,3,59.302,1,"""2021-09-02 23:00:00""",1,5851,58
15,1,1,0.0,0,"""2021-09-02 23:00:00""",1,5852,59
15,1,1,21.756,1,"""2021-09-02 23:00:00""",1,5853,59


In [13]:
dfs["forecast_weather"].filter(
    (pl.col("latitude") == 57.6) &
    (pl.col("longitude") == 21.7) &
    (pl.col("origin_datetime") == "2021-09-01 02:00:00")
).select(
    ["latitude", "longitude", "hours_ahead", "origin_datetime"]
).group_by("origin_datetime").agg(pl.col("hours_ahead").mean())

origin_datetime,hours_ahead
str,f64
"""2021-09-01 02:00:00""",24.5


In [14]:
agg_columns = [col for col in dfs["forecast_weather"].columns 
               if col not in ["latitude", "longitude"] + 
               ["forecast_datetime", 'data_block_id']]
agg_dict = {agg_col: ["mean"] for agg_col in agg_columns}

dfs["forecast_weather"].group_by(["forecast_datetime", 'data_block_id']).agg(
    pl.mean(agg_columns)
)

forecast_datetime,data_block_id,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2021-11-05 02:00:00""",65,null,25.0,7.517538,6.483776,0.374459,0.788241,0.930931,0.999334,-5.107638,2.407442,0.0,0.0,0.0,0.000828
"""2023-01-13 01:00:00""",498,null,48.0,2.547195,1.822729,0.230212,0.996794,0.714138,0.99985,1.465805,6.375389,0.0,0.0,0.000116,0.000515
"""2022-11-07 06:00:00""",432,null,29.0,7.805334,6.237741,0.9839,0.646853,0.686397,0.9998,3.204765,5.962434,0.0,0.0,0.0,0.000066
"""2021-11-26 22:00:00""",86,null,45.0,0.462178,-1.622852,0.68024,0.548291,0.894366,0.961348,-3.388509,-1.414764,0.0,0.0,0.000109,0.000138
"""2023-04-06 15:00:00""",583,null,13.0,7.781676,3.271873,0.371023,0.161743,0.121847,0.429985,-3.997157,-1.128954,631.914921,456.694643,0.0,6.4926e-7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-02-25 21:00:00""",542,null,44.0,-3.086759,-5.321074,0.739758,0.942859,0.748293,0.955175,-3.73673,-7.943222,0.0,0.0,0.000057,0.000057
"""2023-02-27 15:00:00""",544,null,38.0,-2.958223,-5.219591,0.231138,0.465095,0.008772,0.525808,3.614254,1.136308,414.168333,183.839881,1.5966e-7,7.8763e-8
"""2022-06-18 06:00:00""",290,null,28.0,14.00735,12.099392,0.999698,0.017041,0.60023,0.999998,1.996138,3.770625,77.048254,108.461587,0.0,0.000002


In [15]:
dfs["forecast_weather"].head()


latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
f64,f64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,str,f64,f64,f64,f64
57.6,21.7,"""2021-09-01 02:00:00""",1,15.655786,11.553613,0.904816,0.019714,0.0,0.905899,-0.411328,-9.106137,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0
57.6,22.2,"""2021-09-01 02:00:00""",1,13.003931,10.689844,0.886322,0.004456,0.0,0.886658,0.206347,-5.355405,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0
57.6,22.7,"""2021-09-01 02:00:00""",1,14.206567,11.671777,0.729034,0.005615,0.0,0.730499,1.451587,-7.417905,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0
57.6,23.2,"""2021-09-01 02:00:00""",1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0
57.6,23.7,"""2021-09-01 02:00:00""",1,15.293848,12.458887,0.102875,0.088074,0.000015,0.17659,1.268481,-8.975766,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0


In [32]:
result = dfs["train"].with_columns(
    pl.col("datetime").str.strptime(
        pl.Datetime, "%Y-%m-%d %H:%M:%S"
    )
)

In [33]:
result

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,datetime[μs],i64,i64,i64
0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
0,0,1,96.59,1,2021-09-01 00:00:00,0,1,0
0,0,2,0.0,0,2021-09-01 00:00:00,0,2,1
0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
…,…,…,…,…,…,…,…,…
15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64
15,1,1,0.0,0,2023-05-31 23:00:00,637,2018348,59
15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59


In [42]:
result = result.with_columns([
    pl.col("datetime").dt.truncate("1d").cast(pl.Date).alias("date"),
    pl.col("datetime").dt.year().alias("year")
])

In [24]:
dfa = pl.DataFrame({
    "A":[1, 3, 4, 3, 2],
    "B":["uno", "tres", "cuatro", "tres", "dos"]
})
dfa

A,B
i64,str
1,"""uno"""
3,"""tres"""
4,"""cuatro"""
3,"""tres"""
2,"""dos"""


In [25]:
dfb = pl.DataFrame({
    "A":[1, 2, 3, 4],
    "C":["one", "two", "three", "four"]
})
dfb

A,C
i64,str
1,"""one"""
2,"""two"""
3,"""three"""
4,"""four"""


In [26]:
dfa.join(dfb, how="left", on="A")

A,B,C
i64,str,str
1,"""uno""","""one"""
3,"""tres""","""three"""
4,"""cuatro""","""four"""
3,"""tres""","""three"""
2,"""dos""","""two"""


In [ ]:
# PASAR A RADIANES
# HIVERSINE DISTANCE
# JOIN CON LOS COUNTIES MAS CERCANOS